This is my scrapbook. I wrote code here to work through my thought process, try and find patterns in the HTML for item 1, try bs4, regex, sec-api and more. It also contains the start of a function that could be used in a script to query the 10-k.htm links to validate my proof of concept with further efforts. 

In [1]:
# Import packages
from sec_edgar_downloader import Downloader
from bs4 import BeautifulSoup
import re
import pandas as pd
import requests
from sec_api import ExtractorApi
from sec_api import QueryApi

In [ ]:
# start of a script to query the htm links for a companies 10-k 
# written with Lucene query language
# currently returns a json table of values, one of which is the HTM link
# Currently returns 10-k and sometimes extra 10-k/a values

queryApi = QueryApi("b33c5215f9268b5f8356340e5452cff0e52d23d9a52d01e5a956409948f60bb6")

query = {
    "query": {
        "query_string": {
            "query": "formType:\"10-K\" AND ticker:(V)"
        }
    },
    "from": "0",
    "size": "30",
    "sort": [{ "filedAt": { "order": "desc" } }]
}

filings = queryApi.get_filings(query)
print(filings)

In [ ]:
# function to download the 10-k txt and HTML file locally for a given company
# creates folder titled "given_ticker" with subfolders for each year
# folders found inside "sec-edgar-fillings"

def getFile(company,):
    dl = Downloader()
    dl.get("10-K", company)
getFile("AAPL")

In [5]:
# function to download only the 10-k txt file locally for a given company
# creates folder titled "given_ticker" with subfolders for each year
# folders found inside "sec-edgar-fillings"
def getFileTXT(company):
    dl = Downloader()
    dl.get("10-K", company, download_details = False)
getFileTXT("V")

In [ ]:
# using bs4 to try and search for item 1 within the 10-k
# item 1 has minimal tags so hard to find outside of the 10-k index
soup = BeautifulSoup(open("test.html", encoding="utf8"), "html.parser")

print(soup.find_all( "a", string = "Item 1. "))

In [ ]:
# using bs4 with a lxml parser to look for patters in the 10-k html
# using search arguement "body" to try and narrow scope

soup = BeautifulSoup(open("test.html", encoding="utf8"), "lxml")
x = soup.find("body")
print(x.prettify())

In [ ]:
# using bs4 with a html parser to look for patters in the 10-k html

soup = BeautifulSoup(open("test.html", encoding="utf8"), "html.parser")
print(soup.prettify())


In [ ]:
# using bs4 and re to try and search, method notworking sucssesful

f = open("test.txt", "r")
source = f.read()
f.close()
soup = BeautifulSoup(source, "lxml")
print(soup.find_all(string = re.compile("part 1.") ))

In [15]:
# Start of a brute force method to truncate the txt file 
# I stopped as this would not work - it would truncate necesary HTML
# this would make the HTML object incomplete

f = open("test.txt", "r")
lines = f.readlines()
f.close()
w = open("test_write.txt","w")
countp1 = 0
countpA = 0
sp1 = "item 1."
spA = "item 1a"
for line in lines:
    if(sp1 in line):
        sp1+=1
    if(countp1 == 2 and countpA ==1):
        w.write(line)
    if(spA in line):
        countpA+=1
    if(countp1 == 2 and countpA ==2):
        break
w.close()

In [ ]:
# inital attempts to use sec-api to parse the 10-k
# url must be the sec.gov URL, not a local website

extractorApi = ExtractorApi("b33c5215f9268b5f8356340e5452cff0e52d23d9a52d01e5a956409948f60bb6")
url = "https://www.sec.gov/Archives/edgar/data/320193/000119312511282113/d220209d10k.htm"
secH = extractorApi.get_section(url, "1", "html")
print(secH)

In [ ]:
# used this cell to debug my function to write the json file for a dictionary

dictionary = createDictionary("TSLA.txt")
print(dictionary)
writeJSON(dictionary,"TSLA")
#f = open("TSLA.json", "w")
#json.dump(dictionary, f)